## Assignment

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

    to add enough Markdown cells to explain what you decided to do and to report any observations you make.
    to generate maps to visualize your neighborhoods and how they cluster together. 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda update -n base -c defaults conda -y # update Conda

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    sci

<a id='item1'></a>

## 1. Scrape Wiki Page List of Toronto Neighborhoods

Use beautifulsoup to scrape Wiki (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [3]:
!pip install beautifulsoup4

import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

check = False
while check is False:
    try:
        check = True
        html = urllib.request.urlopen(url).read()
    except:
        check = False
        print("Error")
        url = input("Enter site: ")
    continue

soup = BeautifulSoup(html, "html.parser")

#print(soup.prettify())

# Retrieve all of the anchor tags
tags = soup('td')

scraped_data = list()
for tag in tags:
    try:
        #print('TAG:', tag.contents[0].string)
        word = str(tag.contents[0].string)
        scraped_data.append(word)
            
    except: break

count=0 
postal_code = list()
borough = list()
neighborhood = list()
for item in scraped_data:
    if count==0: 
        postal_code.append(item)
        count=1
    elif count==1:
        borough.append(item)
        count=2
    elif count==2:
        item = item.rstrip()
        neighborhood.append(item)
        count=0

#print(postal_code)
#print(borough)
#print(neighborhood)


zipped_list = list(zip(postal_code, borough, neighborhood))
#zipped_list

     |████████████████████████████████| 112kB 4.2MB/s eta 0:00:01


In [30]:
import pandas as pd
import re

neighborhoods = pd.DataFrame(zipped_list, columns = ['Postal Code' , 'Borough', 'Neighborhood']) 

# Drop rows if borough is 'Not assigned'. Reset index.

index_bor = neighborhoods[ neighborhoods['Borough'] == 'Not assigned' ].index
 
neighborhoods.drop(index_bor, inplace=True)

neighborhoods.reset_index(drop=True, inplace=True)

# Replace neighborhood name with name of borough if neighborhood not assigned

neighborhoods['duplicates'] = pd.Series([0 for x in range(len(neighborhoods.index))], index=neighborhoods.index)

count=0
for item in neighborhoods['Neighborhood']:
    #print(item)
    if item=='Not assigned':
        neighborhoods['Neighborhood'][count] = neighborhoods['Borough'][count]
    
    try:
        if neighborhoods['Postal Code'][count] == neighborhoods['Postal Code'][count+1]:
            newvar = neighborhoods['Neighborhood'][count] + ', ' + neighborhoods['Neighborhood'][count+1]
            neighborhoods['Neighborhood'][count] = newvar
            neighborhoods['duplicates'][count+1] = 1
    except: continue
    
    count=count+1

# Drop duplicate Postal Codes

new_index = neighborhoods[ neighborhoods['duplicates'] == 1 ].index
neighborhoods.drop(new_index , inplace=True)
neighborhoods.reset_index(drop=True, inplace=True)
neighborhoods.drop(['duplicates'], axis=1, inplace=True)

neighborhoods.head(20)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.shape


The dataframe has 11 boroughs and 103 neighborhoods.


(103, 3)

#### Import geospatial data for each postal code and merge with dataframe.

In [113]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url2 = 'https://cocl.us/Geospatial_data'

check = False
while check is False:
    try:
        check = True
        html = urllib.request.urlopen(url).read()
    except:
        check = False
        print("Error")
        url = input("Enter site: ")
    continue
    

geodf = pd.read_csv(url2)
geodf.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [114]:
torontodf = pd.merge(neighborhoods, geodf, how='inner', on='Postal Code', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

torontodf.sort_values(by=['Latitude'], inplace=True)
torontodf.reset_index(drop=True, inplace=True)

#torontodf.head(12)

#### Use geopy library to get the latitude and longitude values of Toronto.

In [116]:
import geopy

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [117]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontodf['Latitude'], torontodf['Longitude'], torontodf['Borough'], torontodf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

## Define Foursquare Credentials and Version

In [190]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


## Explore neighborhoods in Toronto

In [119]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
toronto_venues = getNearbyVenues(names=torontodf['Neighborhood'],
                                   latitudes=torontodf['Latitude'],
                                   longitudes=torontodf['Longitude']
                                  )

Alderwood, Long Branch
Humber Bay Shores, Mimico South
Kingsway Park South West, Mimico NW
CN Tower, Bathurst Quay
Humber Bay, King's Mill Park
Brockton, Exhibition Place
Canada Post Gateway Processing Centre
Harbourfront East, Toronto Islands
Bloordale Gardens, Eringate
Berczy Park
Stn A PO Boxes 25 The Esplanade
Design Exchange, Toronto Dominion Centre
Little Portugal, Trinity
Commerce Court, Victoria Hotel
First Canadian Place, Underground city
Parkdale, Roncesvalles
Adelaide, King
Cloverdale, Islington
St. James Town
Runnymede, Swansea
Chinatown, Grange Park
The Kingsway, Montgomery Road
Harbourfront
Ryerson, Garden District
Central Bay Street
Studio District
High Park, The Junction South
Queen's Park
Harbord, University of Toronto
Business Reply Mail Processing Centre 969 Eastern
Church and Wellesley
Queen's Park
Cabbagetown, St. James Town
The Beaches West, India Bazaar
Dovercourt Village, Dufferin
Christie
The Annex, North Midtown
The Junction North, Runnymede
The Beaches
The Da

In [163]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East",4,4,4,4,4,4
"Albion Gardens, Beaumond Heights",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,57,57,57,57,57,57


In [164]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


In [165]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head(20)

In [166]:
toronto_onehot.shape

(2238, 271)

In [167]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0

In [168]:
toronto_grouped.shape

(99, 271)

## Top 3 venues in each neighborhood

In [169]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King----
         venue  freq
0  Coffee Shop  0.08
1          Bar  0.04
2   Steakhouse  0.04


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25


----Agincourt North, L'Amoreaux East----
         venue  freq
0         Park  0.25
1   Playground  0.25
2  Coffee Shop  0.25


----Albion Gardens, Beaumond Heights----
                  venue  freq
0         Grocery Store   0.2
1              Pharmacy   0.1
2  Fast Food Restaurant   0.1


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1    Skating Rink   0.1
2  Sandwich Place   0.1


----Bathurst Manor, Downsview North----
         venue  freq
0  Coffee Shop  0.10
1     Pharmacy  0.05
2    Gift Shop  0.05


----Bayview Village----
                venue  freq
0                Café  0.25
1                Bank  0.25
2  Chinese Restaurant  0.25


----Bedford Park, Lawrence Manor East----
      

In [170]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [186]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King",Coffee Shop,Steakhouse,Bar,Café,Burger Joint
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Donut Shop
2,"Agincourt North, L'Amoreaux East",Park,Coffee Shop,Asian Restaurant,Playground,Dog Run
3,"Albion Gardens, Beaumond Heights",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Fast Food Restaurant
4,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Athletics & Sports


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhoods into 5 clusters.

In [187]:
# set number of clusters
kclusters = 20

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([ 1, 17, 14, 17, 17,  1,  1,  1,  1,  1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [188]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = torontodf.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged['Cluster Labels'].fillna(value=0, inplace=True)
 
toronto_merged.head(10) # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,17.0,Pizza Place,Pub,Pharmacy,Sandwich Place,Athletics & Sports
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South",43.605647,-79.501321,17.0,Bakery,Gym,Café,Liquor Store,Restaurant
2,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW",43.628841,-79.520999,17.0,Social Club,Sandwich Place,Bakery,Discount Store,Tanning Salon
3,M5V,Downtown Toronto,"CN Tower, Bathurst Quay",43.628947,-79.394420,1.0,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boutique
4,M8Y,Etobicoke,"Humber Bay, King's Mill Park",43.636258,-79.498509,17.0,Construction & Landscaping,Baseball Field,Business Service,Women's Store,Donut Shop
5,M6K,West Toronto,"Brockton, Exhibition Place",43.636847,-79.428191,1.0,Café,Breakfast Spot,Coffee Shop,Burrito Place,Italian Restaurant
6,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,1.0,Hotel,Coffee Shop,American Restaurant,Sandwich Place,Burrito Place
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands",43.640816,-79.381752,1.0,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café
8,M9C,Etobicoke,"Bloordale Gardens, Eringate",43.643515,-79.577201,1.0,Convenience Store,Café,Liquor Store,Pharmacy,Pizza Place
9,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1.0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop


## Create map of clustered neighborhoods in Toronto

In [189]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

#### Cluster 1

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,0.0,Pizza Place,Pub,Pharmacy,Sandwich Place,Athletics & Sports,Pool,Skating Rink,Coffee Shop,Gym,Comfort Food Restaurant
1,Etobicoke,0.0,Bakery,Gym,Café,Liquor Store,Restaurant,Seafood Restaurant,Pharmacy,Pizza Place,Fast Food Restaurant,Fried Chicken Joint
2,Etobicoke,0.0,Social Club,Sandwich Place,Bakery,Discount Store,Tanning Salon,Burrito Place,Burger Joint,Convenience Store,Supplement Shop,Hardware Store
3,Downtown Toronto,0.0,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boutique,Bar,Rental Car Location,Plane,Coffee Shop,Harbor / Marina
4,Etobicoke,0.0,Construction & Landscaping,Baseball Field,Business Service,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
5,West Toronto,0.0,Café,Breakfast Spot,Coffee Shop,Burrito Place,Italian Restaurant,Intersection,Restaurant,Stadium,Bar,Furniture / Home Store
6,Mississauga,0.0,Hotel,Coffee Shop,American Restaurant,Sandwich Place,Burrito Place,Fried Chicken Joint,Middle Eastern Restaurant,Gym,Cosmetics Shop,Convenience Store
7,Downtown Toronto,0.0,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Scenic Lookout,Brewery,Restaurant,Fried Chicken Joint,Music Venue
8,Etobicoke,0.0,Convenience Store,Café,Liquor Store,Pharmacy,Pizza Place,Coffee Shop,Beer Store,Doner Restaurant,Dim Sum Restaurant,Diner
9,Downtown Toronto,0.0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Farmers Market,Steakhouse,Bakery,Beer Bar,Hotel
